<a href="https://colab.research.google.com/github/karthikpuranik11/LoResMT/blob/main/MT_en_mr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# create a seperate folder to store everything
!mkdir finetuning
%cd finetuning

/content/finetuning


In [2]:
# clone the repo for running finetuning
!git clone https://github.com/AI4Bharat/indicTrans.git
%cd indicTrans
# clone requirements repositories
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!git clone https://github.com/rsennrich/subword-nmt.git
%cd ..

Cloning into 'indicTrans'...
remote: Enumerating objects: 428, done.
remote: Counting objects: 100% (428/428), done.
remote: Compressing objects: 100% (295/295), done.
remote: Total 428 (delta 249), reused 254 (delta 128), pack-reused 0
Receiving objects: 100% (428/428), 1.42 MiB | 18.47 MiB/s, done.
Resolving deltas: 100% (249/249), done.
/content/finetuning/indicTrans
Cloning into 'indic_nlp_library'...
remote: Enumerating objects: 1325, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 1325 (delta 84), reused 89 (delta 41), pack-reused 1178
Receiving objects: 100% (1325/1325), 9.57 MiB | 15.65 MiB/s, done.
Resolving deltas: 100% (688/688), done.
Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 133, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 133 (delta 0), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (133/133), 149.7

In [3]:
! sudo apt install tree

# Install the necessary libraries
!pip install sacremoses pandas mock sacrebleu tensorboardX pyarrow indic-nlp-library
# Install fairseq from source
!git clone https://github.com/pytorch/fairseq.git
%cd fairseq
# !git checkout da9eaba12d82b9bfc1442f0e2c6fc1b895f4d35d
!pip install --editable ./
%cd ..

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 40.7 kB of archives.
After this operation, 105 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tree amd64 1.7.0-5 [40.7 kB]
Fetched 40.7 kB in 1s (69.1 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package tree.
(Reading database ... 160772 files and directories currently instal

In [4]:
# download the indictrans model


# downloading the en-indic model
# this will contain:
# en-indic/
# ├── final_bin                          # contains fairseq dictionaries (we will use this to binarize the new finetuning data)
# │   ├── dict.SRC.txt
# │   └── dict.TGT.txt
# ├── model                              # contains model checkpoint(s)
# │   └── checkpoint_best.pt
# └── vocab                              # contains bpes for src and tgt (since we train seperate vocabularies) generated with subword_nmt (we will use this bpes to convert finetuning data to subwords)
#     ├── bpe_codes.32k.SRC
#     ├── bpe_codes.32k.TGT
#     ├── vocab.SRC
#     └── vocab.TGT



!wget https://storage.googleapis.com/samanantar-public/V0.2/models/en-indic.zip
!unzip en-indic.zip

# if you want to finetune indic-en models, use the link below

# !wget https://akpublicdata.blob.core.windows.net/indicnlp/indictrans/indictrans-indic-en-v0.2.zip
# !unzip indictrans-indic-en-v0.2.zip


--2021-06-15 14:20:19--  https://storage.googleapis.com/samanantar-public/V0.2/models/en-indic.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.199.128, 74.125.20.128, 74.125.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.199.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4609212103 (4.3G) [application/zip]
Saving to: ‘en-indic.zip’

en-indic.zip        100%[===================>]   4.29G  60.2MB/s    in 85s     

2021-06-15 14:21:44 (51.6 MB/s) - ‘en-indic.zip’ saved [4609212103/4609212103]

Archive:  en-indic.zip
   creating: en-indic/
   creating: en-indic/vocab/
  inflating: en-indic/vocab/bpe_codes.32k.SRC  
  inflating: en-indic/vocab/vocab.SRC  
  inflating: en-indic/vocab/vocab.TGT  
  inflating: en-indic/vocab/bpe_codes.32k.TGT  
   creating: en-indic/final_bin/
  inflating: en-indic/final_bin/dict.TGT.txt  
  inflating: en-indic/final_bin/dict.SRC.txt  
   creating: en-indic/model/
  infla

In [5]:
!sudo apt-get install p7zip-full

Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-6).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [6]:
!mkdir /content/finetuning/dataset
!mkdir /content/finetuning/dataset/train
!mkdir /content/finetuning/dataset/dev
!mkdir /content/finetuning/dataset/test
!mkdir /content/finetuning/dataset/train/en-mr

In [7]:
%cd /content/

/content


In [8]:
!7z x /content/en_mr.zip
#password loresmt2021@covidmr


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 1667350 bytes (1629 KiB)

Extracting archive: /content/en_mr.zip
--
Path = /content/en_mr.zip
Type = zip
Physical Size = 1667350

  0%    
Enter password (will not be echoed):
  0% - dev.en             Everything is Ok

Files: 4
Size:       8026001
Compressed: 1667350


In [9]:
!cp /content/dev.en /content/finetuning/dataset/dev
!cp /content/dev.mr /content/finetuning/dataset/dev

In [10]:
!mv /content/dev.en /content/finetuning/dataset/test/
!mv /content/dev.mr /content/finetuning/dataset/test/

In [11]:
!mv /content/train.en /content/finetuning/dataset/train/en-mr
!mv /content/train.mr /content/finetuning/dataset/train/en-mr

In [16]:
%cd /content/finetuning/dataset/train/en-mr

/content/finetuning/dataset/train/en-mr


In [18]:
%cat *.en > train1.en

In [19]:
%cd /content/finetuning/indicTrans

/content/finetuning/indicTrans


In [20]:
%%shell

exp_dir=../dataset
src_lang=en
tgt_lang=indic

# change this to indic-en, if you have downloaded the indic-en dir
download_dir=../en-indic

train_data_dir=$exp_dir/train
dev_data_dir=$exp_dir/dev
test_data_dir=$exp_dir/test
echo $exp_dir



../dataset


In [21]:
# all the data preparation happens in this cell
%%shell

exp_dir=../dataset
src_lang=en
tgt_lang=indic

# change this to indic-en, if you have downloaded the indic-en dir
download_dir=../en-indic

train_data_dir=$exp_dir/train
dev_data_dir=$exp_dir/dev
test_data_dir=$exp_dir/test


echo "Running experiment ${exp_dir} on ${src_lang} to ${tgt_lang}"


train_processed_dir=$exp_dir/data
devtest_processed_dir=$exp_dir/data

out_data_dir=$exp_dir/final_bin

mkdir -p $train_processed_dir
mkdir -p $devtest_processed_dir
mkdir -p $out_data_dir

# indic languages.
# cvit-pib corpus does not have as (assamese) and kn (kannada), hence its not part of this list
langs=( mr )

for lang in ${langs[@]};do
	if [ $src_lang == en ]; then
		tgt_lang=$lang
	else
		src_lang=$lang
	fi

	train_norm_dir=$exp_dir/norm/$src_lang-$tgt_lang
	devtest_norm_dir=$exp_dir/norm/$src_lang-$tgt_lang
	mkdir -p $train_norm_dir
	mkdir -p $devtest_norm_dir


    # preprocessing pretokenizes the input (we use moses tokenizer for en and indicnlp lib for indic languages)
    # after pretokenization, we use indicnlp to transliterate all the indic data to devnagiri script

	# train preprocessing
	train_infname_src=$train_data_dir/en-${lang}/train.$src_lang
	train_infname_tgt=$train_data_dir/en-${lang}/train.$tgt_lang
	train_outfname_src=$train_norm_dir/train.$src_lang
	train_outfname_tgt=$train_norm_dir/train.$tgt_lang
	echo "Applying normalization and script conversion for train $lang"
	input_size=`python scripts/preprocess_translate.py $train_infname_src $train_outfname_src $src_lang true`
	input_size=`python scripts/preprocess_translate.py $train_infname_tgt $train_outfname_tgt $tgt_lang true`
	echo "Number of sentences in train $lang: $input_size"

	# dev preprocessing
	dev_infname_src=$dev_data_dir/dev.$src_lang
	dev_infname_tgt=$dev_data_dir/dev.$tgt_lang
	dev_outfname_src=$devtest_norm_dir/dev.$src_lang
	dev_outfname_tgt=$devtest_norm_dir/dev.$tgt_lang
	echo "Applying normalization and script conversion for dev $lang"
	input_size=`python scripts/preprocess_translate.py $dev_infname_src $dev_outfname_src $src_lang true`
	input_size=`python scripts/preprocess_translate.py $dev_infname_tgt $dev_outfname_tgt $tgt_lang true`
	echo "Number of sentences in dev $lang: $input_size"

	# test preprocessing
	test_infname_src=$test_data_dir/test.$src_lang
	test_infname_tgt=$test_data_dir/test.$tgt_lang
	test_outfname_src=$devtest_norm_dir/test.$src_lang
	test_outfname_tgt=$devtest_norm_dir/test.$tgt_lang
	echo "Applying normalization and script conversion for test $lang"
	input_size=`python scripts/preprocess_translate.py $test_infname_src $test_outfname_src $src_lang true`
	input_size=`python scripts/preprocess_translate.py $test_infname_tgt $test_outfname_tgt $tgt_lang true`
	echo "Number of sentences in test $lang: $input_size"
done




# Now that we have preprocessed all the data, we can now merge these different text files into one
# ie. for en-as, we have train.en and corresponding train.as, similarly for en-bn, we have train.en and corresponding train.bn
# now we will concatenate all this into en-X where train.SRC will have all the en (src) training data and train.TGT will have all the concatenated indic lang data

python scripts/concat_joint_data.py $exp_dir/norm $exp_dir/data $src_lang $tgt_lang 'train'
python scripts/concat_joint_data.py $exp_dir/norm $exp_dir/data $src_lang $tgt_lang 'dev'
python scripts/concat_joint_data.py $exp_dir/norm $exp_dir/data $src_lang $tgt_lang 'test'

# use the vocab from downloaded dir
cp -r $download_dir/vocab $exp_dir


echo "Applying bpe to the new finetuning data"
bash apply_single_bpe_traindevtest_notag.sh $exp_dir

mkdir -p $exp_dir/final

# We also add special tags to indicate the source and target language in the inputs
#  Eg: to translate a sentence from english to hindi , the input would be   __src__en__   __tgt__hi__ <en bpe tokens>

echo "Adding language tags"
python scripts/add_joint_tags_translate.py $exp_dir 'train'
python scripts/add_joint_tags_translate.py $exp_dir 'dev'
python scripts/add_joint_tags_translate.py $exp_dir 'test'



data_dir=$exp_dir/final
out_data_dir=$exp_dir/final_bin

rm -rf $out_data_dir

# binarizing the new data (train, dev and test) using dictionary from the download dir

 num_workers=`python -c "import multiprocessing; print(multiprocessing.cpu_count())"`

data_dir=$exp_dir/final
out_data_dir=$exp_dir/final_bin

# rm -rf $out_data_dir

echo "Binarizing data. This will take some time depending on the size of finetuning data"
fairseq-preprocess --source-lang SRC --target-lang TGT \
 --trainpref $data_dir/train --validpref $data_dir/dev --testpref $data_dir/test \
 --destdir $out_data_dir --workers $num_workers \
 --srcdict $download_dir/final_bin/dict.SRC.txt --tgtdict $download_dir/final_bin/dict.TGT.txt --thresholdtgt 5 --thresholdsrc 5  

Running experiment ../dataset on en to indic
Applying normalization and script conversion for train mr
100% 81809/81809 [00:15<00:00, 5291.49it/s]
100% 81809/81809 [00:09<00:00, 8671.00it/s]
Number of sentences in train mr: 81809
Applying normalization and script conversion for dev mr
100% 500/500 [00:00<00:00, 1047.32it/s]
100% 500/500 [00:00<00:00, 4236.94it/s]
Number of sentences in dev mr: 500
Applying normalization and script conversion for test mr
100% 500/500 [00:00<00:00, 1046.96it/s]
100% 500/500 [00:00<00:00, 5120.35it/s]
Number of sentences in test mr: 500

../dataset/data/train.SRC
../dataset/data/train.TGT
  0% 0/11 [00:00<?, ?it/s]src: en, tgt:as
src: en, tgt:bn
src: en, tgt:gu
src: en, tgt:hi
src: en, tgt:kn
src: en, tgt:ml
src: en, tgt:mr
../dataset/norm/en-mr/train.en
../dataset/norm/en-mr/train.mr
src: en, tgt:or
src: en, tgt:pa
src: en, tgt:ta
src: en, tgt:te
100% 11/11 [00:00<00:00, 279.94it/s]
  0% 0/11 [00:00<?, ?it/s]src: en, tgt:as
src: en, tgt:bn
src: en, tgt:g

In [22]:
# Finetuning the model

# pls refer to fairseq documentaion to know more about each of these options (https://fairseq.readthedocs.io/en/latest/command_line_tools.html)


# some notable args:
# --max-update=1000     -> for this example, to demonstrate how to finetune we are only training for 1000 steps, incrase this if needed
# --arch=transformer_4x -> we use a custom transformer model and name it transformer_4x (4 times the parameter size of transformer  base)
# --user_dir            -> we define the custom transformer arch in model_configs folder and pass it as an argument to user_dir for fairseq to register this architechture
# --lr                  -> learning rate. From our limited experiments, we find that lower learning rates like 3e-5 works best for finetuning.
# --restore-file        -> reload the pretrained checkpoint and start training from here (change this path for indic-en. Currently its is set to en-indic)
# --reset-*             -> reset and not use lr scheduler, dataloader, optimizer etc of the older checkpoint
# --max_tokns           -> this is max tokens per batch


!( fairseq-train ../dataset/final_bin \
--max-source-positions=210 \
--max-target-positions=210 \
--max-update=100000 \
--save-interval=1 \
--arch=transformer_4x \
--criterion=label_smoothed_cross_entropy \
--source-lang=SRC \
--max-epoch=3 \
--lr-scheduler=inverse_sqrt \
--target-lang=TGT \
--label-smoothing=0.1 \
--optimizer adam \
--adam-betas "(0.9, 0.98)" \
--clip-norm 1.0 \
--warmup-init-lr 1e-07 \
--warmup-updates 4000 \
--dropout 0.2 \
--tensorboard-logdir ../dataset/tensorboard-wandb \
--save-dir ../dataset/model \
--keep-last-epochs 5 \
--patience 5 \
--skip-invalid-size-inputs-valid-test \
--fp16 \
--no-last-checkpoints \
--user-dir model_configs \
--update-freq=2 \
--distributed-world-size 1 \
--max-tokens 256 \
--lr 3e-5 \
--restore-file ../en-indic/model/checkpoint_best.pt \
--reset-lr-scheduler \
--reset-meters \
--reset-dataloader \
--reset-optimizer)

2021-06-15 14:45:36 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'tensorboard_logdir': '../dataset/tensorboard-wandb', 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': 'model_configs', 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': None,

In [32]:
# To test the models after training, you can use joint_translate.sh



# joint_translate takes src_file, output_fname, src_lang, tgt_lang, model_folder as inputs
# src_file -> input text file to be translated
# output_fname -> name of the output file (will get created) containing the model predictions
# src_lang -> source lang code of the input text ( in this case we are using en-indic model and hence src_lang would be 'en')
# tgt_lang -> target lang code of the input text ( tgt lang for en-indic model would be any of the 11 indic langs we trained on:
#              as, bn, hi, gu, kn, ml, mr, or, pa, ta, te)
# supported languages are:
#              as - assamese, bn - bengali, gu - gujarathi, hi - hindi, kn - kannada, 
#              ml - malayalam, mr - marathi, or - oriya, pa - punjabi, ta - tamil, te - telugu

# model_dir -> the directory containing the model and the vocab files

# Note: if the translation is taking a lot of time, please tune the buffer_size and batch_size parameter for fairseq-interactive defined inside this joint_translate script


# here we are translating the english sentences to hindi
!bash /content/finetuning/indicTrans/joint_translate.sh /content/finetuning/dataset/test/test.en en_mr_outputs.txt 'en' 'mr' /content/finetuning/dataset

Tue Jun 15 17:17:01 UTC 2021
Applying normalization and script conversion
100% 500/500 [00:00<00:00, 1103.10it/s]
Number of sentences in input: 500
Applying BPE
Decoding
Extracting translations, script conversion and detokenization
Translation completed


In [34]:
!cat en_mr_outputs.txt

सिव्हीअर एक्युट रेस्पिरेटरी सिंड्रोम कोरोनाव्हायरस 2 (SARS-CoV-2) हा एक नवीन विषाणू आहे, जो सर्वप्रथम वुहानमधील श्वसनाच्या तीव्र आजाराच्या प्रकरणांच्या समूहाशी संबंधित न्यूमोनियाच्या तीन रुग्णांपासून वेगळा करण्यात आला.
गुरुवारी, सार्स-सीओव्ही-२ कोरोना विषाणूच्या संसर्गाच्या जगभरातील एकूण प्रकरणांची संख्या दहा लाखांच्या वर गेल्याचं जॉन्स हॉपकिन्स युनिव्हर्सिटीच्या डेटामधून सूचित करण्यात आलं.
सर्दी झालेल्या रुग्णांच्या नाकातील स्त्रावापासून एचसीओव्ही-२२९ई स्ट्रेन बी८१४ ला पहिल्यांदा वेगळे करण्यापूर्वी विविध संक्रमित प्राण्यांमध्ये वेगवेगळ्या सीओव्ही वेगळे करण्यात आले होते ज्यात टर्की, उंदीर, गाय, डुक्कर, मांजर आणि कुत्रा यांचा समावेश होता.
फेलिन कोरोनाव्हायरस (FCoV) मांजरांमध्ये सौम्य एन्टेरायटीस तसेच तीव्र फेलिन संसर्गजन्य पेरीटोनिटीस (त्याच विषाणूचे इतर प्रकार) निर्माण करतो.
चिनी सरकारचे वरिष्ठ वैद्यकीय सल्लागार झोंग नानशान ह्यांनी असा युक्तीवाद केला की जर सर्व देशांना WHO च्या विषाणूचा प्रसार रोखण्याच्या उपायांचा सल्ला पाळण्यासाठी एकत्र आणता आले तर "जून महिन्यापर्यंत हे संपुष्टात येऊ 

In [35]:
# to compute bleu scores for the predicitions with a reference file, use the following command
# arguments:
# pred_fname: file that contains model predictions
# ref_fname: file that contains references
# src_lang and tgt_lang : the source and target language

!bash /content/finetuning/indicTrans/compute_bleu.sh en_mr_outputs.txt /content/finetuning/dataset/test/test.mr 'en' 'mr'


100% 500/500 [00:00<00:00, 6235.27it/s]
100% 500/500 [00:00<00:00, 6274.41it/s]
BLEU+case.mixed+numrefs.1+smooth.exp+tok.none+version.1.5.1 = 27.9 58.6/34.7/22.3/14.9 (BP = 0.972 ratio = 0.972 hyp_len = 12576 ref_len = 12934)


In [36]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [37]:
!cp -r /content/finetuning/dataset /content/drive/MyDrive/en-mr/finetuning